<a href="https://colab.research.google.com/github/kytk/MagiciansCorner/blob/master/MedNISTClassify.ipynb?hl=ja" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MedNISTデータセットを用いた放射線画像の分類
# Classify the type of radiological image using the MedNIST collection

### Bradley J Erickson, MD PhD
*Copyright 2019

### このNotebookは、Radiology: AI article の以下の論文に対応しています
### See corresponding Radiology: AI article here:
https://pubs.rsna.org/doi/10.1148/ryai.2019190072


このチュートリアルでは以下の3つを行います:

1) 6種類の画像をダウンロードし、展開します (頭部CT, 胸部CT, 腹部CT, 頭部MR, 乳腺MR, 胸部Xp) 
2) 写真を用いて事前にトレーニングされた畳み込みニューラルネットワーク (CNN) と ResNet 34 アーキテクチャを用いて画像を3種類に分類します 
3) システムの性能を評価し、一番間違っている結果を記録し、どのように性能を改善できるか考慮します 

In this tutorial we will: 1) download and unzip 6 different classes of radiological images (CT Head, CT Chest, CT Abdomen, MRBrain, MRBreast, and Chest X-Ray). 2) We will use a Convolutional Neural Network (CNN) pretrained on routine photgraphic images and using the ResNet 34 architecture to classify each of the 3 types of images 3) We will review the performance of the system, note the 'worst' errors, and consider how we might improve performance


In [ ]:
# セル 1
# 最初に、fastai ライブラリをインストールしたうえで、必要なモジュールをインポートします
# first, need to install and then import the fastai library
!pip3 install fastai
from fastai.vision import *

In [ ]:
# セル 2
# 再度セルを実行する時の為に、念の為に以前のデータを削除します
# clean out any old data just to be sure, such as if re-running cells
!rm -rf MagiciansCorner
!rm -rf images

!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mqgBKTB0MtGf8Fhc8HaedJyiD8yMoXOh' -O ./MedNIST.zip

!mkdir images
!cd images; unzip -q "../MedNIST.zip" 
!rm -rf MagiciansCorner
# macOSによって生成された不要なファイルを削除します
# get rid of MAC garbage stuff
!rm -rf ./images/__MACOSX
!ls images


In [ ]:
# セル 3
classes_dir = "./images"
flist = os.listdir(classes_dir)
print (flist)

* 事前にDICOM画像をグレイスケールのJPEG画像に変換し、サイズを 64x64 にしてあります (もし胸部レントゲン画像を 64x64 以上のサイズのままにしていたら、畳み込みニューラルネットワークの実行に失敗します。）すべての画像が同じサイズでないといけません。次の論文でこのことについてもう少し説明します。

    * We have already gone through and converted the DICOM images to JPEG (grayscale) images, and also sized so they are all 64x64 (if we kept CXRs as some size other than 64x64, it would cause the CNN to fail--all image MUST be the same size. The next article will explain more about this.

## データを表示する
## View data

In [ ]:
# セル 4
np.random.seed(42)
data = ImageDataBunch.from_folder(classes_dir, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=64, num_workers=4).normalize(imagenet_stats)
data.classes
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

* いいですね。それではいくつかの画像を見てみましょう。

    * Good! Let's take a look at some of our pictures then.


In [ ]:
# セル 5
data.show_batch(rows=3, figsize=(7,8))

def get_img(img_url): return open_image(img_url)

# 画像を様々なtransformationから表示する関数
# Function that displays many transformations of an image
def plots_of_one_image(img_url, tfms, rows=1, cols=3, width=15, height=5, **kwargs):
    img = get_img(img_url)
    [img.apply_tfms(tfms, **kwargs).show(ax=ax)
         for i,ax in enumerate(plt.subplots(rows,cols,figsize=(width,height))[1].flatten())]
tfms = get_transforms(flip_vert=False,                # flip vertical and horizontal 垂直方向と水平方向の反転
                      max_rotate=20.0,                # rotation between -30° and 30°-30°と30°の間で回転
                      max_zoom=1.2)                   # zoom between 1 and 1.2　1-1.2の間で拡大
# Uncomment the line below to turn off augmentation (sets the transformations to nothing. Note that you will still see many images, but they are all the same
# tfms=[[],[]]

# Uncomment these 3 lines to show examples of artificial/augmented images from 1 starting image
# note that 00000124.jpg is my randomly selected head CT
# all displayed images are variants of that 1 image
#plots_of_one_image('./images/MRBrain/00000129.jpg',tfms[0],9,14,11,7, size=64)
#plt.subplots_adjust(left=0, bottom=0,wspace=0, hspace=0)
#plt.show()

## モデルの学習
## Train model

In [ ]:
# セル 6
learn = cnn_learner(data, models.resnet34, metrics=error_rate)
#learn = cnn_learner(data, models.resnet50, metrics=error_rate)


In [ ]:
# セル 7
learn.fit_one_cycle(3)
learn.save("MedNIST-34-1")

## 評価
## Evaluation

* 学習の過程で、データは3パートにわけられます。学習、テスト、検証。学習データは重み付けを調整するために使われます。GPUは学習に使う画像すべてを一度に読み込めるだけのメモリがないので、「バッチ」に分けられます。学習のためのすべての画像が使われると、「エポック」が1回完了したことになります。1エポックが終わると、どのくらいモデルが学習できたかを「テスト」データを用いて評価します。学習データにおける性能は train_loss で、検証データにおける性能は valid_loss であらわします。error_rate は、検証データでどの程度間違えたかの割合を示します。
    * During the training process, the data is split into 3 parts: training, testing and validation. The training data is used to adjust the weights. The GPU does not have enough RAM to store the entire training set of images, so it is split into 'batches'. When all of the images have been used once for training, then an 'epoch' has passed. Once trained for that epoch, it evaluates how well it has learned using the 'test' data set. The performance on the training set is the train_loss and the performance on the validation set is the valid_loss, and the error_rate is also the percentage of cases wrong in the validation set.

* 通常、検証データセットでいい性能が得られたら、テストデータを使って実際の性能がどうかを評価します。
    * It is common practice that after 'acceptable' performance is achieved on the validation set, that the system is tested on the 'test' data, and that is what is considered the 'real' performance.

* 注意していただきたいのは、人によってはこのテストデータを使うことを「検証(validation)」と言うことです。
    * Note that some use 'test' for what is called validation here, and vice versa.

* しかし時々、全体的なエラーだけでは何が起こっているかをつかむことが難しいかもしれません。偽陰性よりも偽陽性に注意する必要があるかもしれませんし、その逆かもしれません。最初の頃の結果をよくみることで、価値のある発見をすることができ、そのうえで学習に進むことで結果が改善します。
    * But sometimes the overall error rate doesn't really tell the story. We might care more about false positives than false negatives, and vica versa. Looking at early results can provide valuable insight into the training process, and how to improve results.


In [ ]:
# セル 8
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

## もっとよく見てみる
## Looking closer

* confusion matrix を見ると、胸部と腹部は、頭部と比べると混乱が起きています。驚きますか？
    * The confusion matrix shows that there is more confusion between chests and abdomens than with heads. Does that surprise you?

* これらをもう少し詳しくみていきましょう。FastAIにはもっともよくなかった結果を表示する関数があります。それについて考えてみましょう。間違いがあるわけですが、もっともよくなかった間違いは何でしょうか？
    * Let's look a little closer at those. FastAI has a nice function that can show you the cases the it did the worst on. Think about that--there are 'errors' but what are the worst errors?

* ある画像にが適切なクラスに割り当てられると、スコアがもっともよくなります。従って、最低なものというのは、正しいクラスにおけるスコアがもっとも低いものです。'plot_top_losses' 関数は想定されるクラス、真実のクラス、スコア、をワーストN位まで（今の場合は9）表示します。
    * Well, the class assigned to an image is the class that gets the highest score. So the 'worst' would be those where the score for the correct class was lowest. The function 'plot_top_losses' will show the predicted class, the real class, and the score, as well as the image for the N 
(in our case, 9) worst scored cases.

* セル9にあるコードの2行目は、fastAIの便利な機能を示しています。関数についての説明を見たい時に、'doc(関数)' とタイプすればその関数に関してのドキュメントを表示します。ドキュメントの中にあるリンクをクリックすると、ソースコードを表示します。
    * The second line of code in the cell shows another nice feature of FastAI: to get documentation on any function, just type 'doc(function)' and it will print the documentation for that function. AND it also has a link you can click to then see the actual source code that implements that function.


In [ ]:
# セル 9
interp.plot_top_losses(9, figsize=(10,10))
doc(interp.plot_top_losses)

## 何がわかりますか？
## What do we see?

* エラーとなったクラスの多くは肺と腹部のどちらもが含まれているスライスです。
    * Most of the errant classes are slices that contain BOTH lung and abdomen. 
* これは重要なポイントです。データの準備と整理はよい結果を得るために必要不可欠です。
    * This is an important point: Data preparation and curation is critical to getting good results
* このような例に対する対処は悩むところです。正解はケースバイケースです。ポイントは、このようなエラーになった例をきちんと見てみない限り、何が悪かったのかを知ることはできないということです。
    * We can argue about how to handle these cases. The correct answer probably depends on your use case. The point is that without seeing these error cases, you might never know what was going wrong...


In [ ]:
# セル 10
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()


## おまけ
## Extra credit:
* 100万以上もの画像を使って既に学習が終わっているネットワークを使ってずるしてみました。
    * We 'cheated' by starting with a network that was already trained on more than 1,000,000 images. That means the system really only had to learn the specific features of these body parts, but the lower level features like edges and lines were already 'known' to be important to the network.

* 一方、このすでに学習されたネットワークに使われたデータはグレースケールではなく、カラー写真であり、64x64以上のマトリクスです。
    * On the other hand, the 'pretrained' network was trained on photographic images, which are color, not gray scale, and had a matrix size other than 64x64. 

* ゼロからはじめることもできますが、オプションとして、事前に学習で得られた値を使うものの、重み付けやカーネルは変更可能にするという使い方ができます。これが'unfreeze'が行うことです。
    * While we could start from scratch, a better option might be to use the pre-trained values, but allow any of the weights and kernels in the network to be changed, and that is what 'unfreeze' does. 


In [ ]:
#Cell 1
learn.fit_one_cycle(5, max_lr=slice(3e-6,3e-5))
learn.save("Unfreeze-34-1")